# DEPLOYMENT SYSTEM TIMESERIES

#Mengambil dan membaca data NO2 sumenep

Kode di bawah digunakan untuk membaca file CSV berisi data kadar NO₂ menggunakan pustaka pandas di Python. Pertama, path file didefinisikan dalam variabel DATA_PATH, kemudian fungsi pd.read_csv() digunakan untuk memuat data ke dalam sebuah DataFrame bernama df. Program menampilkan lima baris pertama dataset menggunakan df.head(), informasi struktur data (seperti jumlah kolom, tipe data, dan nilai non-null) dengan df.info(), serta ringkasan statistik deskriptif seperti rata-rata, standar deviasi, dan nilai maksimum menggunakan df.describe(). Blok try-except digunakan untuk menangani kemungkinan kesalahan seperti file tidak ditemukan atau format CSV yang tidak sesuai, sehingga program tidak langsung berhenti saat terjadi error.

In [2]:
import pandas as pd

# Path ke file CSV
DATA_PATH = "/content/NO2_Sumenep.csv"

try:
    # Membaca data CSV
    df = pd.read_csv(DATA_PATH)

    # Tampilkan beberapa baris awal
    print("📊 Data Awal NO₂ Sumenep:")
    print(df.head())

    # Tampilkan informasi umum dataset
    print("\n🧭 Info Dataset:")
    print(df.info())

    # Tampilkan statistik dasar
    print("\n📈 Deskripsi Statistik:")
    print(df.describe())

except FileNotFoundError:
    print(f" File tidak ditemukan di path: {DATA_PATH}")
except Exception as e:
    print(f" Terjadi kesalahan saat membaca CSV: {e}")


📊 Data Awal NO₂ Sumenep:
                  time       NO2
0  2023-10-15 00:00:00  0.000007
1  2023-10-16 00:00:00  0.000013
2  2023-10-17 00:00:00  0.000018
3  2023-10-18 00:00:00  0.000014
4  2023-10-19 00:00:00  0.000014

🧭 Info Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    728 non-null    object 
 1   NO2     624 non-null    float64
dtypes: float64(1), object(1)
memory usage: 11.5+ KB
None

📈 Deskripsi Statistik:
              NO2
count  624.000000
mean     0.000014
std      0.000006
min     -0.000017
25%      0.000010
50%      0.000013
75%      0.000017
max      0.000043


#Membuat prediksi model knn regressor
Kode di bawah menggunakan algoritma K-Nearest Neighbors (KNN) untuk melakukan prediksi deret waktu (time series) kadar NO₂. Fungsi train_knn_model() bertugas melatih model KNN berdasarkan data latih (X_train dan y_train) dengan parameter n_neighbors yang menentukan jumlah tetangga terdekat yang digunakan dalam prediksi. Model menggunakan pembobotan weights='distance', yang berarti tetangga yang lebih dekat memiliki pengaruh lebih besar dalam menentukan hasil prediksi. Setelah pelatihan, model siap digunakan untuk memprediksi nilai di luar data latih (future forecasting).

Fungsi forecast_next_days() digunakan untuk memprediksi nilai NO₂ beberapa hari ke depan (n_future). Proses dimulai dengan mengambil data input terakhir dari X_test, kemudian secara iteratif memprediksi nilai berikutnya dan memperbarui input untuk prediksi selanjutnya dengan cara menggeser (np.roll) dan menambahkan nilai prediksi terbaru di posisi terakhir. Hasil prediksi disimpan dalam future_preds, lalu dibuat DataFrame forecast_df yang berisi tanggal-tanggal masa depan (future_dates) serta nilai prediksi NO₂ yang sesuai. Dengan demikian, fungsi ini menghasilkan output yang siap digunakan untuk analisis atau visualisasi tren polutan di masa depan.

In [ ]:
from sklearn import neighbors
import pandas as pd
import numpy as np

def train_knn_model(X_train, y_train, n_neighbors=5):
    model = neighbors.KNeighborsRegressor(n_neighbors=n_neighbors, weights='distance')
    model.fit(X_train, y_train)
    return model

def forecast_next_days(model, X_test, df, scaler, n_future):
    last_input = X_test[-1, :].reshape(1, -1)
    future_preds = []

    for _ in range(n_future):
        next_pred = model.predict(last_input)[0]
        future_preds.append(next_pred)
        last_input = np.roll(last_input, -1)
        last_input[0, -1] = next_pred

    future_dates = pd.date_range(df.index[-1] + pd.Timedelta(days=1), periods=n_future)
    forecast_df = pd.DataFrame({"Tanggal": future_dates, "Prediksi_NO2": np.array(future_preds)})
    return forecast_df


#Pemrosesan dan Pembentukan Data
Kode di bawah berfungsi untuk mempersiapkan data deret waktu (time series) kadar NO₂ agar siap digunakan dalam pemodelan. Fungsi load_and_prepare_data() bertugas membaca file CSV yang berisi data pengukuran NO₂ dengan kolom waktu (time) dan nilai kadar NO₂ (NO2). Kolom waktu dikonversi menjadi tipe datetime agar bisa digunakan sebagai indeks deret waktu, kemudian dilakukan interpolasi dengan metode waktu (interpolate(method='time')) untuk mengisi nilai yang hilang berdasarkan tren waktu. Langkah ini memastikan bahwa data memiliki urutan kronologis yang konsisten dan bebas dari nilai kosong yang bisa mengganggu pelatihan model.

Fungsi create_supervised_data() digunakan untuk mengubah data deret waktu menjadi bentuk supervised learning, yaitu dengan membuat sejumlah fitur lag (NO2(t-1), NO2(t-2), dan seterusnya) sebagai input untuk memprediksi nilai NO2(t) di hari berikutnya. Data kemudian dibagi menjadi data latih (70%) dan data uji (30%) untuk evaluasi model. Sebelum digunakan dalam algoritma KNN, seluruh fitur diubah ke dalam rentang [0, 1] menggunakan MinMaxScaler, agar setiap variabel memiliki skala yang seimbang dan model dapat belajar lebih efektif. Hasil akhirnya adalah data latih dan uji yang telah ditransformasi, siap untuk digunakan dalam proses pelatihan model prediksi.


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def load_and_prepare_data(path):
    df = pd.read_csv(path)
    df['time'] = pd.to_datetime(df['time'], errors='coerce')
    df = df.set_index('time')
    df['NO2'] = df['NO2'].interpolate(method='time')
    return df

def create_supervised_data(df, n_lags):
    supervised = pd.DataFrame()
    for i in range(n_lags, 0, -1):
        supervised[f'NO2(t-{i})'] = df['NO2'].shift(i)
    supervised['NO2(t)'] = df['NO2']
    supervised = supervised.dropna()

    X = supervised.drop('NO2(t)', axis=1)
    y = supervised['NO2(t)']

    split_idx = int(len(X) * 0.7)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    scaler = MinMaxScaler(feature_range=(0, 1))
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test, scaler


#Fungsi Forecasting pada Time Series
Kode di bawah berfungsi untuk menghasilkan prediksi kadar NO₂ untuk beberapa hari ke depan menggunakan model time series berbasis KNN. Fungsi forecast_next_days() memulai proses dengan mengambil data input terakhir (last_input) dari data uji (X_test) sebagai kondisi awal prediksi. Model kemudian digunakan secara iteratif untuk memprediksi satu nilai di masa depan, menambahkannya ke daftar hasil (preds), dan memperbarui jendela input menggunakan pendekatan sliding window — di mana nilai prediksi terbaru menggantikan nilai lag tertua. Proses ini diulang sebanyak jumlah hari yang ingin diprediksi (n_future), sehingga diperoleh serangkaian nilai prediksi jangka pendek ke depan.

Setelah semua prediksi diperoleh, hasilnya perlu dikembalikan ke skala asli karena sebelumnya data telah dinormalisasi. Oleh karena itu, dilakukan inverse transform menggunakan nilai minimum dan maksimum dari objek MinMaxScaler. Nilai-nilai prediksi kemudian disusun menjadi sebuah DataFrame berisi kolom tanggal (Tanggal) dan hasil prediksi (Prediksi_NO2). Tanggal-tanggal tersebut dihasilkan secara otomatis berdasarkan tanggal terakhir pada dataset historis dengan frekuensi harian. Hasil akhirnya berupa tabel berisi tanggal-tanggal masa depan beserta kadar NO₂ yang diprediksi, yang dapat digunakan untuk visualisasi tren atau analisis lanjutan.

In [ ]:
# utils/forecast_utils.py
import numpy as np
import pandas as pd

def forecast_next_days(model, X_test, df, scaler, n_future):
    """
    Membuat prediksi untuk n hari ke depan menggunakan model time series.
    """
    last_input = X_test[-1].reshape(1, -1)
    preds = []

    for _ in range(n_future):
        pred = model.predict(last_input)
        pred_value = pred[0] if pred.ndim == 1 else pred[0, 0]
        preds.append(pred_value)

        # sliding window
        new_input = np.append(last_input[:, 1:], pred_value).reshape(1, -1)
        last_input = new_input

    # --- inverse transform hanya kolom target (NO2) ---
    min_val = scaler.data_min_[-1]  # ambil min dari kolom target
    max_val = scaler.data_max_[-1]  # ambil max dari kolom target

    preds_rescaled = [p * (max_val - min_val) + min_val for p in preds]

    # buat DataFrame hasil forecast (kolom disesuaikan dengan plot_utils)
    last_date = df.index[-1]
    forecast_dates = pd.date_range(last_date, periods=n_future + 1, freq='D')[1:]
    forecast_df = pd.DataFrame({
        'Tanggal': forecast_dates,
        'Prediksi_NO2': preds_rescaled
    })

    return forecast_df


#Evaluasi Model (Metrics Utils)
Kode di bawah berfungsi untuk mengevaluasi kinerja model regresi time series, khususnya model KNN, dengan menghitung berbagai metrik statistik. Fungsi evaluate_model() menerima tiga parameter utama: model yang telah dilatih (model), data uji (X_test), dan nilai aktual (y_test). Fungsi ini kemudian menghasilkan prediksi (y_pred) dengan memanggil metode .predict() dari model. Setelah prediksi diperoleh, tiga metrik utama digunakan untuk mengukur performa model, yaitu Root Mean Squared Error (RMSE), R-squared (R²), dan Mean Absolute Percentage Error (MAPE).

RMSE digunakan untuk mengukur seberapa besar kesalahan rata-rata dalam satuan yang sama dengan data aslinya, di mana nilai yang lebih kecil menunjukkan hasil prediksi yang lebih akurat. R² mengukur seberapa baik model mampu menjelaskan variabilitas data aktual — nilai mendekati 1 berarti model sangat baik dalam menjelaskan data. Sementara itu, MAPE menampilkan kesalahan rata-rata dalam bentuk persentase, memudahkan interpretasi terhadap seberapa besar deviasi prediksi terhadap nilai aktual. Hasil akhir fungsi ini mengembalikan keempat nilai tersebut (y_pred, rmse, r2, mape) yang kemudian dapat digunakan untuk pelaporan, perbandingan model, atau analisis performa prediksi.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
import numpy as np

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    return y_pred, rmse, r2, mape


#Visualisasi Hasil Prediksi (Plot Utils)
Kode di bawah berisi tiga fungsi utama untuk visualisasi hasil prediksi model KNN pada data time series NO₂. Fungsi plot_actual_vs_pred() digunakan untuk membandingkan antara nilai aktual (y_test) dan hasil prediksi (y_pred) dari model. Grafik ini menampilkan dua garis dengan warna berbeda untuk menunjukkan seberapa baik model mengikuti pola data asli. Parameter k menunjukkan jumlah tetangga (neighbors) pada algoritma KNN, yang ditampilkan dalam judul grafik agar hasil dapat lebih mudah dianalisis dan dibandingkan.

Selanjutnya, fungsi plot_forecast() menampilkan grafik prediksi ke depan (forecasting) berdasarkan data historis dan hasil peramalan beberapa hari berikutnya. Data historis divisualisasikan sebagai garis abu-abu, sedangkan hasil prediksi ditunjukkan dengan garis merah berbentuk titik-titik untuk membedakan masa depan dari data sebelumnya. Fungsi terakhir, plot_residual_acf(), digunakan untuk menganalisis autokorelasi residual, yaitu selisih antara nilai aktual dan prediksi. Dengan menggunakan fungsi plot_acf() dari statsmodels, grafik ini membantu mengevaluasi apakah masih ada pola dalam residual — jika tidak, berarti model sudah cukup baik dalam menangkap pola data. Ketiga fungsi ini sangat penting untuk memvalidasi dan mempresentasikan performa model secara visual.

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf

def plot_actual_vs_pred(y_test, y_pred, k):
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(y_test.index, y_test, label='Aktual', color='darkorange')
    ax.plot(y_test.index, y_pred, label='Prediksi', color='navy')
    ax.set_title(f"KNN Forecasting NO₂ (K={k})")
    ax.set_xlabel("Tanggal")
    ax.set_ylabel("Kadar NO₂")
    ax.legend()
    return fig

def plot_forecast(df, forecast_df):
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(df.index, df['NO2'], label="Data Historis", color='gray')
    ax.plot(forecast_df['Tanggal'], forecast_df['Prediksi_NO2'], 'ro-', label="Prediksi NO₂")
    ax.set_title("Prediksi NO₂ Beberapa Hari ke Depan")
    ax.legend()
    return fig

def plot_residual_acf(y_test, y_pred):
    residuals = y_test - y_pred
    fig, ax = plt.subplots(figsize=(8, 3))
    plot_acf(residuals, lags=20, ax=ax)
    ax.set_title("Autokorelasi Residual")
    return fig


#Aplikasi Streamlit untuk Prediksi NO₂ Sumenep Menggunakan KNN Time Series
Kode di bawah merupakan aplikasi Streamlit interaktif yang digunakan untuk memprediksi kadar gas NO₂ (Nitrogen Dioksida) di wilayah Sumenep menggunakan metode K-Nearest Neighbors (KNN) berbasis time series forecasting. Aplikasi ini membaca data dari file NO2_Sumenep.csv, memprosesnya menjadi format terstruktur menggunakan fungsi load_and_prepare_data(), dan menampilkan grafik historis kadar NO₂. Pengguna dapat mengatur parameter model secara dinamis melalui sidebar, seperti jumlah lag (hari sebelumnya yang digunakan sebagai input), jumlah tetangga (neighbors), dan jumlah hari ke depan yang ingin diprediksi.

Setelah data dipersiapkan dan model KNN dilatih, aplikasi menampilkan hasil evaluasi performa model melalui metrik seperti RMSE (Root Mean Square Error), R² (Koefisien Determinasi), dan MAPE (Mean Absolute Percentage Error). Hasil prediksi divisualisasikan dalam grafik yang membandingkan data aktual dengan hasil prediksi serta proyeksi kadar NO₂ untuk beberapa hari ke depan. Selain itu, disediakan juga analisis autokorelasi residual untuk menilai kualitas model. Pengguna dapat mengunduh hasil prediksi dalam format CSV langsung dari antarmuka. Dengan desain modular dan antarmuka interaktif, aplikasi ini sangat membantu dalam eksplorasi, analisis, serta pemantauan kualitas udara secara prediktif.

In [ ]:
import sys, os
sys.path.append(os.path.dirname(__file__))

import streamlit as st
import pandas as pd
from statsmodels.graphics.tsaplots import plot_acf

from utils.data_utils import load_and_prepare_data, create_supervised_data
from utils.plot_utils import plot_actual_vs_pred, plot_forecast, plot_residual_acf
from utils.metrics_utils import evaluate_model
from utils.forecast_utils import forecast_next_days
from models.knn_model import train_knn_model


# --- Konfigurasi Tampilan ---
st.set_page_config(page_title="Prediksi NO₂ Sumenep", layout="wide")
st.title("🌫️ Prediksi Kadar NO₂ Sumenep Menggunakan KNN Time Series")

# --- Baca dataset lokal ---
DATA_PATH = r"data\NO2_Sumenep.csv"

try:
    df = load_and_prepare_data(DATA_PATH)
except Exception as e:
    st.error(f"Gagal membaca dataset: {e}")
    st.stop()

# --- Tampilkan data awal ---
st.subheader("📈 Data Historis NO₂")
st.line_chart(df['NO2'])

# --- Sidebar pengaturan model ---
st.sidebar.header("⚙️ Pengaturan Model")
n_lags = st.sidebar.slider("Jumlah Lag Hari Sebelumnya", 3, 14, 7)
n_neighbors = st.sidebar.slider("Jumlah Tetangga (K)", 2, 15, 5)
n_future = st.sidebar.slider("Hari Prediksi ke Depan", 1, 14, 7)

# --- Persiapan data supervised ---
X_train, X_test, y_train, y_test, scaler = create_supervised_data(df, n_lags)

# --- Training model KNN ---
model = train_knn_model(X_train, y_train, n_neighbors)

# --- Evaluasi model ---
y_pred, rmse, r2, mape = evaluate_model(model, X_test, y_test)

st.subheader("📊 Evaluasi Model")
st.write(f"**RMSE:** {rmse:.6f}")
st.write(f"**R²:** {r2:.4f}")
st.write(f"**MAPE:** {mape:.2f}%")

st.pyplot(plot_actual_vs_pred(y_test, y_pred, n_neighbors))

# --- Prediksi ke depan ---
st.subheader(f"🔮 Prediksi {n_future} Hari ke Depan")
forecast_df = forecast_next_days(model, X_test, df, scaler, n_future)
st.dataframe(forecast_df)
st.pyplot(plot_forecast(df, forecast_df))

# --- Analisis residual ---
st.subheader("🔍 Analisis Autokorelasi Residual")
st.pyplot(plot_residual_acf(y_test, y_pred))

# --- Tombol unduh hasil ---
csv = forecast_df.to_csv(index=False).encode('utf-8')
st.download_button("⬇️ Unduh Hasil Prediksi (CSV)", csv, "forecast_no2.csv", "text/csv")
